In [19]:
import pandas as pd

First, we will perform data exploration and cleaning on the original dataset, as
well as feature engineering to enhance and obtain new features.

In [20]:
# Display all rows and columns

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [21]:
# Read in base data from csv

df = pd.read_csv('nfl.csv')
df.head(10)

,Rk,Team,Date,Season,Pts,PtsO,Rate,TO,Y/P,DY/P,ToP,Rate.1,Att,Att.1,Day,G#,Week,Away,Opp,Result,Pts.1,PtsO.1,PtDif,PC,Cmp,Att.2,Inc,Cmp%,Yds,TD,Int,TD%,Int%,Rate.2,Sk,Yds.1,Sk%,Y/A,NY/A,AY/A,ANY/A,Y/C,Rush_Att,Rush_Yds,Rush_Y/A,TD.1,Tot,Ply,Y/P.1,DPly,DY/P.1,TO.1,ToP.1,Time,Cmp.1,Att.3,Cmp%.1,Yds.2,TD.2,Sk.1,Yds.3,Int.1,Rate.3,Opp_Rush_Att,Opp_Rush_Yds,Y/A.1,TD.3,Rush,Pass,Tot.1,TO.2
0,1,DET,2024-12-05,2024,34,31,109.7,0,5.14,6.62,36:06,110.2,34,24,Thu,13,14,NaN,GNB,W 34-31,34,31,3,65,32,41,9,78.0,280,3,1,7.3,2.4,109.7,1,3,2.38,6.8,6.67,7.20,7.02,8.8,34,111,3.3,1,391,76,5.14,45,6.62,1,36:06,3:12,12,20,60.0,199,1,1,7,0,110.2,24,99,4.1,3,12,81,93,0
1,2,GNB,2024-12-05,2024,31,34,111.7,0,6.62,5.14,23:54,109.3,24,34,Thu,13,14,@,DET,L 31-34,31,34,-3,65,12,20,8,60.0,199,1,0,5.0,0.0,111.7,1,7,4.76,10.0,9.48,10.95,10.43,16.6,24,99,4.1,3,298,45,6.62,76,5.14,1,23:54,3:12,32,41,78.0,280,3,1,3,1,109.3,34,111,3.3,1,-12,-81,-93,0
2,3,DEN,2024-12-02,2024,41,32,65.7,1,6.56,6.57,27:50,86.5,26,23,Mon,13,13,NaN,CLE,W 41-32,41,32,9,73,18,35,17,51.4,294,1,2,2.9,5.7,65.7,0,0,0.00,8.4,8.40,6.40,6.40,16.3,26,106,4.1,2,400,61,6.56,84,6.57,2,27:50,3:29,34,58,58.6,475,4,3,22,3,86.5,23,77,3.3,0,29,-181,-152,1
3,4,CLE,2024-12-02,2024,32,41,88.1,-1,6.57,6.56,32:10,65.7,23,26,Mon,12,13,@,DEN,L 32-41,32,41,-9,73,34,58,24,58.6,475,4,3,6.9,5.2,88.1,3,22,4.92,8.2,7.79,7.24,6.89,14.0,23,77,3.3,0,552,84,6.57,61,6.56,3,32:10,3:29,18,35,51.4,294,1,0,0,2,65.7,26,106,4.1,2,-29,181,152,-1
4,5,JAX,2024-12-01,2024,20,23,83.0,-1,5.57,5.34,27:43,92.5,25,25,Sun,12,13,NaN,HOU,L 20-23,20,23,-3,43,24,42,18,57.1,276,2,1,4.8,2.4,83.0,0,0,0.00,6.6,6.57,6.45,6.45,11.5,25,97,3.9,0,373,67,5.57,61,5.34,1,27:43,3:18,22,34,64.7,218,1,2,24,0,92.5,25,108,4.3,1,-11,58,47,-1
5,6,CIN,2024-12-01,2024,38,44,112.7,-2,6.58,7.88,28:51,126.0,15,26,Sun,12,13,NaN,PIT,L 38-44,38,44,-6,82,28,38,10,73.7,282,3,1,7.9,2.6,112.7,4,27,9.52,7.4,6.71,7.82,7.07,10.1,15,93,6.2,1,375,57,6.58,66,7.88,3,28:51,3:20,29,38,76.3,410,3,2,4,1,126.0,26,110,4.2,1,-17,-128,-145,-2
6,7,TAM,2024-12-01,2024,26,23,70.7,-1,5.78,5.40,39:31,82.6,39,21,Sun,12,13,@,CAR,W 26-23 (OT),26,23,3,49,22,34,12,64.7,209,1,2,2.9,5.9,70.7,4,31,10.53,6.1,5.50,4.09,3.66,9.5,39,236,6.1,1,445,77,5.78,68,5.40,2,39:31,3:38,26,46,56.5,289,1,1,9,0,82.6,21,78,3.7,1,158,-80,78,-1
7,8,SFO,2024-12-01,2024,10,35,74.8,-3,5.09,6.64,26:33,138.9,27,38,Sun,12,13,@,BUF,L 10-35,10,35,-25,45,11,18,7,61.1,86,0,0,0.0,0.0,74.8,2,8,10.00,4.8,4.30,4.78,4.30,7.8,27,153,5.7,1,239,47,5.09,56,6.64,3,26:33,2:46,14,18,77.8,152,2,0,0,0,138.9,38,220,5.8,3,-67,-66,-133,-3
8,9,LAR,2024-12-01,2024,21,14,110.2,0,5.85,4.81,27:23,85.9,29,31,Sun,12,13,@,NOR,W 21-14,21,14,7,35,14,24,10,58.3,166,2,0,8.3,0.0,110.2,2,17,7.69,6.9,6.38,8.58,7.92,11.9,29,156,5.4,1,322,55,5.85,68,4.81,0,27:23,2:53,24,37,64.9,184,1,0,0,0,85.9,31,143,4.6,0,13,-18,-5,0
9,10,ATL,2024-12-01,2024,13,17,40.0,-3,4.55,4.07,35:55,83.9,37,17,Sun,12,13,NaN,LAC,L 13-17,13,17,-4,30,24,39,15,61.5,234,0,4,0.0,10.3,40.0,1,11,2.50,6.0,5.85,1.38,1.35,9.8,37,116,3.1,1,350,77,4.55,46,4.07,4,35:55,2:44,17,24,70.8,131,0,5,19,0,83.9,17,56,3.3,0,60,103,163,-3


In [22]:
# FEATURE ENGINEERING: Add new columns for team 2
#
# We take the absolute values of the turnover, passing yards, and rushing yards,
# and total yards margins of team 1 to reflect those same margins for team 2.

df['Team2PtDiff'] = -(df['PtDif'])
df['Team2TM'] = -(df['TO'])
df['Team2RYM'] = -(df['Rush'])
df['Team2PYM'] = -(df['Pass'])
df['Team2YM'] = -(df['Tot.1'])

df.head()

,Rk,Team,Date,Season,Pts,PtsO,Rate,TO,Y/P,DY/P,ToP,Rate.1,Att,Att.1,Day,G#,Week,Away,Opp,Result,Pts.1,PtsO.1,PtDif,PC,Cmp,Att.2,Inc,Cmp%,Yds,TD,Int,TD%,Int%,Rate.2,Sk,Yds.1,Sk%,Y/A,NY/A,AY/A,ANY/A,Y/C,Rush_Att,Rush_Yds,Rush_Y/A,TD.1,Tot,Ply,Y/P.1,DPly,DY/P.1,TO.1,ToP.1,Time,Cmp.1,Att.3,Cmp%.1,Yds.2,TD.2,Sk.1,Yds.3,Int.1,Rate.3,Opp_Rush_Att,Opp_Rush_Yds,Y/A.1,TD.3,Rush,Pass,Tot.1,TO.2,Team2PtDiff,Team2TM,Team2RYM,Team2PYM,Team2YM
0,1,DET,2024-12-05,2024,34,31,109.7,0,5.14,6.62,36:06,110.2,34,24,Thu,13,14,NaN,GNB,W 34-31,34,31,3,65,32,41,9,78.0,280,3,1,7.3,2.4,109.7,1,3,2.38,6.8,6.67,7.20,7.02,8.8,34,111,3.3,1,391,76,5.14,45,6.62,1,36:06,3:12,12,20,60.0,199,1,1,7,0,110.2,24,99,4.1,3,12,81,93,0,-3,0,-12,-81,-93
1,2,GNB,2024-12-05,2024,31,34,111.7,0,6.62,5.14,23:54,109.3,24,34,Thu,13,14,@,DET,L 31-34,31,34,-3,65,12,20,8,60.0,199,1,0,5.0,0.0,111.7,1,7,4.76,10.0,9.48,10.95,10.43,16.6,24,99,4.1,3,298,45,6.62,76,5.14,1,23:54,3:12,32,41,78.0,280,3,1,3,1,109.3,34,111,3.3,1,-12,-81,-93,0,3,0,12,81,93
2,3,DEN,2024-12-02,2024,41,32,65.7,1,6.56,6.57,27:50,86.5,26,23,Mon,13,13,NaN,CLE,W 41-32,41,32,9,73,18,35,17,51.4,294,1,2,2.9,5.7,65.7,0,0,0.00,8.4,8.40,6.40,6.40,16.3,26,106,4.1,2,400,61,6.56,84,6.57,2,27:50,3:29,34,58,58.6,475,4,3,22,3,86.5,23,77,3.3,0,29,-181,-152,1,-9,-1,-29,181,152
3,4,CLE,2024-12-02,2024,32,41,88.1,-1,6.57,6.56,32:10,65.7,23,26,Mon,12,13,@,DEN,L 32-41,32,41,-9,73,34,58,24,58.6,475,4,3,6.9,5.2,88.1,3,22,4.92,8.2,7.79,7.24,6.89,14.0,23,77,3.3,0,552,84,6.57,61,6.56,3,32:10,3:29,18,35,51.4,294,1,0,0,2,65.7,26,106,4.1,2,-29,181,152,-1,9,1,29,-181,-152
4,5,JAX,2024-12-01,2024,20,23,83.0,-1,5.57,5.34,27:43,92.5,25,25,Sun,12,13,NaN,HOU,L 20-23,20,23,-3,43,24,42,18,57.1,276,2,1,4.8,2.4,83.0,0,0,0.00,6.6,6.57,6.45,6.45,11.5,25,97,3.9,0,373,67,5.57,61,5.34,1,27:43,3:18,22,34,64.7,218,1,2,24,0,92.5,25,108,4.3,1,-11,58,47,-1,3,1,11,-58,-47


In [23]:
# Remove unnecessary columns and rename
# TODO: explanation of our decision to keep certain columns

columns_to_keep = {
  'Rk': 'ID',
  'Result':'Team1Won',
  'Season': 'Season',
  'Date': 'Date',
  'Team': 'Team1',
  'Opp': 'Team2',
  'Away': 'Home',

  'Pts': 'Team1Pts',
  'PtsO': 'Team2Pts',
  'PtDif': 'Team1PtDiff',
  'Team2PtDiff': 'Team2PtDiff',
  'TO': 'Team1TM',
  'Team2TM': 'Team2TM',
  'Rate': 'Team1Rating',
  'Rate.1': 'Team2Rating',
  # 'Y/P': 'Team1Y',
  # 'Y/P.1': 'Team2Y',
  # 'DY/P': 'Team1DY',
  # 'DY/P.1': 'Team2DY', 
  # 'ToP': 'Team1ToP',
  # 'ToP.1': 'Team2ToP',
  'Sk': 'Team1Sks',
  'Sk.1': 'Team2Sks',
  'Yds.1': 'Team1SkYds',
  'Yds.3': 'Team2SkYds',
  'Rush_Att': 'Team1RushAtt',
  'Opp_Rush_Att': 'Team2RushAtt',
  'Rush_Yds': 'Team1RushYds',
  'Opp_Rush_Yds': 'Team2RushYds',
  'Rush': 'Team1RYM',
  'Team2RYM': 'Team2RYM',
  'Pass': 'Team1PYM',
  'Team2PYM': 'Team2PYM',
  'Tot.1': 'Team1YM',
  'Team2YM': 'Team2YM',
}

trimmed_df = df[list(columns_to_keep.keys())].rename(columns=columns_to_keep)
trimmed_df.head(10)
print(trimmed_df.shape)

(1478, 29)


In [24]:
# Remove duplicate columns

trimmed_df['team_pair'] = trimmed_df.apply(lambda row: frozenset([row['Team1'], row['Team2']]), axis=1)
trimmed_df.drop_duplicates(subset=['Date', 'team_pair'], inplace=True)
trimmed_df.drop(columns=['team_pair'], inplace=True)
trimmed_df.head()

print(trimmed_df.shape)

(739, 29)


In [25]:
# Check for null values

trimmed_df.isna().any()

# The 'Home' column has nulls since a null value means Team 1 is home.
# This will be changed to a boolean below.

ID              False
Team1Won        False
Season          False
Date            False
Team1           False
Team2           False
Home             True
Team1Pts        False
Team2Pts        False
Team1PtDiff     False
Team2PtDiff     False
Team1TM         False
Team2TM         False
Team1Rating     False
Team2Rating     False
Team1Sks        False
Team2Sks        False
Team1SkYds      False
Team2SkYds      False
Team1RushAtt    False
Team2RushAtt    False
Team1RushYds    False
Team2RushYds    False
Team1RYM        False
Team2RYM        False
Team1PYM        False
Team2PYM        False
Team1YM         False
Team2YM         False
dtype: bool

In [26]:
# Let's include a cell here to check the columns that we've decided to include
# in the trimmed down version of the dataset.

trimmed_df.head(10)

,ID,Team1Won,Season,Date,Team1,Team2,Home,Team1Pts,Team2Pts,Team1PtDiff,Team2PtDiff,Team1TM,Team2TM,Team1Rating,Team2Rating,Team1Sks,Team2Sks,Team1SkYds,Team2SkYds,Team1RushAtt,Team2RushAtt,Team1RushYds,Team2RushYds,Team1RYM,Team2RYM,Team1PYM,Team2PYM,Team1YM,Team2YM
0,1,W 34-31,2024,2024-12-05,DET,GNB,NaN,34,31,3,-3,0,0,109.7,110.2,1,1,3,7,34,24,111,99,12,-12,81,-81,93,-93
2,3,W 41-32,2024,2024-12-02,DEN,CLE,NaN,41,32,9,-9,1,-1,65.7,86.5,0,3,0,22,26,23,106,77,29,-29,-181,181,-152,152
4,5,L 20-23,2024,2024-12-01,JAX,HOU,NaN,20,23,-3,3,-1,1,83.0,92.5,0,2,0,24,25,25,97,108,-11,11,58,-58,47,-47
5,6,L 38-44,2024,2024-12-01,CIN,PIT,NaN,38,44,-6,6,-2,2,112.7,126.0,4,2,27,4,15,26,93,110,-17,17,-128,128,-145,145
6,7,W 26-23 (OT),2024,2024-12-01,TAM,CAR,@,26,23,3,-3,-1,1,70.7,82.6,4,1,31,9,39,21,236,78,158,-158,-80,80,78,-78
7,8,L 10-35,2024,2024-12-01,SFO,BUF,@,10,35,-25,25,-3,3,74.8,138.9,2,0,8,0,27,38,153,220,-67,67,-66,66,-133,133
8,9,W 21-14,2024,2024-12-01,LAR,NOR,@,21,14,7,-7,0,0,110.2,85.9,2,0,17,0,29,31,156,143,13,-13,-18,18,-5,5
9,10,L 13-17,2024,2024-12-01,ATL,LAC,NaN,13,17,-4,4,-3,3,40.0,83.9,1,5,11,19,37,17,116,56,60,-60,103,-103,163,-163
10,11,W 25-24,2024,2024-12-01,IND,NWE,@,25,24,1,-1,-1,1,55.7,94.7,0,4,0,16,34,31,144,200,-56,56,-113,113,-169,169
12,13,L 21-26,2024,2024-12-01,NYJ,SEA,NaN,21,26,-5,5,0,0,73.1,90.1,2,3,16,31,21,23,89,84,5,-5,-6,6,-1,1


In [27]:
# Now, let's perform further data cleaning by setting the data types of all 
# columns for consistency in our dataframe.

# We set 'Home' to boolean values to signify whether a game record corresponds 
# to a home game or an away game.

# We also set 'Team1Won' to booleans to represent the result of each game.

from datetime import datetime

formatted_df = trimmed_df
formatted_df['Date'] = formatted_df['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

numeric_columns = ['ID',
				   'Season',
				   'Team1Pts',    
					 'Team2Pts',    
					 'Team1PtDiff',
					 'Team2PtDiff', 
					 'Team1TM',    
					 'Team2TM',     
					 'Team1Rating', 
					 'Team2Rating', 
					 'Team1Sks',    
					 'Team2Sks',    
					 'Team1SkYds',  
					 'Team2SkYds',  
					 'Team1RushAtt',
					 'Team2RushAtt',
					 'Team1RushYds',
					 'Team2RushYds',
					 'Team1RYM',    
					 'Team2RYM',    
					 'Team1PYM',    
					 'Team2PYM',    
					 'Team1YM',     
					 'Team2YM']

str_columns = ['Team1',
			   'Team2']

for column in numeric_columns:
	formatted_df[column] = pd.to_numeric(formatted_df[column], errors='coerce')

for column in str_columns:
	formatted_df[column] = formatted_df[column].astype(str)

formatted_df['Home'] = formatted_df['Home'].isnull()

formatted_df['Team1Won'] = formatted_df['Team1Won'].apply(lambda x: True if str(x).startswith('W') else False)

for column in formatted_df:
	print(type(formatted_df[column][0]))

<class 'numpy.int64'>
<class 'numpy.bool_'>
<class 'numpy.int64'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'str'>
<class 'str'>
<class 'numpy.bool_'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>


In [28]:
# TODO: Additional feature engineering
# TODO: Take the differences of Team1Rating
# Team2Rating     
# Team1Sks        
# Team2Sks        
# Team1SkYds      
# Team2SkYds      
# Team1RushAtt    
# Team2RushAtt
# Team1RushYds
# Team2RushYds
# and set them as margins

Now, we will train the model.

In [29]:
formatted_df = formatted_df.sort_values('Date')
formatted_df = formatted_df.reset_index(drop=True)
formatted_df.head(10)

,ID,Team1Won,Season,Date,Team1,Team2,Home,Team1Pts,Team2Pts,Team1PtDiff,Team2PtDiff,Team1TM,Team2TM,Team1Rating,Team2Rating,Team1Sks,Team2Sks,Team1SkYds,Team2SkYds,Team1RushAtt,Team2RushAtt,Team1RushYds,Team2RushYds,Team1RYM,Team2RYM,Team1PYM,Team2PYM,Team1YM,Team2YM
0,1477,True,2022,2022-09-08,BUF,LAR,False,31,10,21,-21,-1,1,112.0,58.1,2,7,5,49,25,18,121,52,69,-69,101,-101,170,-170
1,1449,False,2022,2022-09-11,ATL,NOR,True,26,27,-1,1,-1,1,79.7,106.7,0,4,0,35,38,19,201,151,50,-50,-19,19,31,-31
2,1451,False,2022,2022-09-11,CAR,CLE,True,24,26,-2,2,-1,1,84.6,72.9,4,1,28,9,19,39,54,217,-163,163,69,-69,-94,94
3,1453,True,2022,2022-09-11,CHI,SFO,True,19,10,9,-9,1,-1,85.7,49.0,2,2,16,9,37,37,99,176,-77,77,-50,50,-127,127
4,1455,False,2022,2022-09-11,CIN,PIT,True,20,23,-3,3,-5,5,61.7,78.0,7,1,39,2,34,22,133,75,58,-58,107,-107,165,-165
5,1457,False,2022,2022-09-11,ARI,KAN,True,21,44,-23,23,1,-1,94.7,144.2,3,0,26,0,22,27,103,128,-25,25,-181,181,-206,206
6,1459,False,2022,2022-09-11,DAL,TAM,True,3,19,-16,16,0,0,53.5,84.6,4,2,25,17,18,33,71,152,-81,81,-22,22,-103,103
7,1475,False,2022,2022-09-11,JAX,WAS,False,22,28,-6,6,2,-2,75.0,100.2,2,1,15,8,18,28,123,85,38,-38,-45,45,-7,7
8,1463,False,2022,2022-09-11,IND,HOU,False,20,20,0,0,-1,1,83.1,96.9,2,3,12,18,38,28,177,77,100,-100,118,-118,218,-218
9,1473,False,2022,2022-09-11,LVR,LAC,False,19,24,-5,5,-3,3,69.1,129.4,6,0,39,0,13,31,64,76,-12,12,-23,23,-35,35


In [30]:
def add_target(team):
  team['Team1Target'] = team['Team1Won'].shift(-1)
  return team

formatted_df = formatted_df.groupby('Team1', group_keys=False).apply(add_target)

C:\Users\13462\AppData\Local\Temp\ipykernel_11548\1349788793.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  formatted_df = formatted_df.groupby('Team1', group_keys=False).apply(add_target)


In [31]:
# Verify that our data is correct: that the number of losses is roughly
# equivalent to the total number of wins.

# TODO: Explain why the counts might be different

print(formatted_df['Team1Target'].value_counts())

print(formatted_df['Team1Won'].value_counts())

Team1Target
False    354
True     353
Name: count, dtype: int64
Team1Won
False    373
True     366
Name: count, dtype: int64


In [32]:
# Verify that Team1Target is doing what it should after grouping by teams

formatted_df[formatted_df['Team1'] == 'DAL'].head()

,ID,Team1Won,Season,Date,Team1,Team2,Home,Team1Pts,Team2Pts,Team1PtDiff,Team2PtDiff,Team1TM,Team2TM,Team1Rating,Team2Rating,Team1Sks,Team2Sks,Team1SkYds,Team2SkYds,Team1RushAtt,Team2RushAtt,Team1RushYds,Team2RushYds,Team1RYM,Team2RYM,Team1PYM,Team2PYM,Team1YM,Team2YM,Team1Target
6,1459,False,2022,2022-09-11,DAL,TAM,True,3,19,-16,16,0,0,53.5,84.6,4,2,25,17,18,33,71,152,-81,81,-22,22,-103,103,True
47,1383,True,2022,2022-09-26,DAL,NYG,False,23,16,7,-7,1,-1,98.2,54.9,0,5,0,27,30,25,176,167,9,-9,46,-46,55,-55,True
53,1359,True,2022,2022-10-02,DAL,WAS,True,25,10,15,-15,2,-2,107.5,55.2,1,2,6,15,29,27,62,142,-80,80,62,-62,-18,18,True
74,1341,True,2022,2022-10-09,DAL,LAR,False,22,10,12,-12,3,-3,80.7,82.1,3,5,26,35,34,15,163,38,125,-125,-209,209,-84,84,False
88,1309,False,2022,2022-10-16,DAL,PHI,False,17,26,-9,9,-3,3,37.3,100.7,0,4,0,23,26,39,134,136,-2,2,49,-49,47,-47,True


In [33]:
exclude = ['ID', 'Team1Won', 'Season', 'Date', 'Team1', 'Team2', 'Team1Target', 'Home']
include = formatted_df.columns[~formatted_df.columns.isin(exclude)]

In [34]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector

from sklearn.preprocessing import StandardScaler

# TODO: Do we put this in the cross-validation loop? If so, how?
scaler = StandardScaler()
formatted_df[include] = scaler.fit_transform(formatted_df[include])

# Prep df for SFS
formatted_df = formatted_df.dropna()
formatted_df['Team1Target'] = formatted_df['Team1Target'].astype(int)

# TODO: n_splits is a hyperparameter to analyze within CV loop
# TODO: n_features_to_select is a hyperparameter to analyze within CV loop

def get_optimal_features(model, n_splits, n_features_to_select):
	split = TimeSeriesSplit(n_splits=n_splits)
	sfs = SequentialFeatureSelector(model, n_features_to_select=n_features_to_select, direction='forward', cv=split)
	sfs.fit(formatted_df[include], formatted_df['Team1Target'])
	return list(include[sfs.get_support()])

In [35]:
# Output the class labels predicted by a certain model, which will be trained 
# based on the top predictors of that model.

def backtest(data, predictors, model, target='Team1Target'):
	results = []
	
	seasons = sorted(data['Season'].unique())
	
	for i in range(2, len(seasons)):
		cur_season = seasons[i]
		train = data[data['Season'] < cur_season]
		test = data[data['Season'] == cur_season]
		
		model.fit(train[predictors], train[target])
		
		pred = model.predict(test[predictors])
		pred = pd.Series(pred, index=test.index)
		
		season_results = pd.DataFrame({
				'Season': cur_season,
				'Actual': test[target].values,
				'Prediction': pred
		})
		results.append(season_results)    
	return pd.concat(results, ignore_index=True)

In [36]:
# How well do different models predict the training set (rolling window size 1)?

# TODO: Note that this is NOT testing. Testing requires the use of a test set to 
# verify the model trained on the training data.

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score

dtc = DecisionTreeClassifier()
lr = LogisticRegression()
rfc = RandomForestClassifier()

models = [dtc, lr, rfc]

for model in models:
	predictors = get_optimal_features(model, 3, 10)
	predictions = backtest(formatted_df, predictors, model)
	print(accuracy_score(predictions['Actual'], predictions['Prediction']))

0.4146341463414634
0.5426829268292683


KeyboardInterrupt: 

In [ ]:
# rolling average

rolling_cols = formatted_df.select_dtypes(include=['number']).columns
 
df_rolling = formatted_df[list(include) + ["Team1Won", 'Team1','Season']]
